In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# loading dataframe
df = pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [ ]:
df.head()

****EDA****

In [ ]:
df.info()

In [ ]:
## cleaning missing values
df.shape

In [ ]:
df[df.duplicated()] ## no duplicates

In [ ]:
#df['ever_married'] = df['ever_married'].map({'Yes': 1, 'No': 0})

In [ ]:
df['gender'].value_counts()

In [ ]:
#df['gender'] = df['gender'].replace({'Female': 1, 'Male': 0, 'Other': 2})
# 2 - Other
# 1 - Female
# 0 - Male

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.work_type.value_counts()

In [ ]:
df.Residence_type.value_counts()
# almost equal number of people from rural and urban areas in the 

In [ ]:
df['avg_glucose_level'].describe()

In [ ]:
df['bmi'].describe()

In [ ]:
df['smoking_status'].value_counts()

In [ ]:
df['stroke'].value_counts()

In [ ]:
## handling missing data - BMI - by Prediction through decision tree - variables used: age and gneder


from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder



DT_bmi_pipeline = Pipeline(steps = [
    ('scale', StandardScaler()),
    ('lr', DecisionTreeRegressor(random_state = 42))
])


X = df[['age', 'gender', 'bmi']].copy()
X.gender = X.gender.replace({'Male': 0, 'Female': 1, 'Other': -1})

X.gender = X.gender.astype(np.uint8)


In [ ]:
## Separting those values that have Missing BMI

# test set would contain all bmi missing 201 values
Missing = X[X.bmi.isna()]

# train set would contain all non-missing bmi values
X_train = X[~X.bmi.isna()]
Y_train = X_train.pop('bmi')

# Fit regressor
DT_bmi_pipeline.fit(X_train, Y_train)

In [ ]:
## Predicting missing BMI and substituting in the original dataset
predicted_bmi = pd.Series(DT_bmi_pipeline.predict(Missing[['age', 'gender']]), index = Missing.index)
predicted_bmi

In [ ]:
# substituting in original dataset
df.loc[Missing.index, 'bmi'] = predicted_bmi

In [ ]:
df.isnull().sum()

## PLOTS TO UNDERSTAND CORRELATIONS

In [ ]:
df.corr(method = 'pearson')

In [ ]:
variables = [variable for variable in df.columns]

conts = ['age','avg_glucose_level','bmi']

In [ ]:
variables

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 12), facecolor='#f6f6f6')
gs = fig.add_gridspec(4, 3)
gs.update(wspace=0.1, hspace=0.4)
gs

In [ ]:
corr = df.corr()

In [ ]:
mask = np.triu(np.ones_like(corr, dtype=bool))
mask

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

import seaborn as sns
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

## Age, hypertension, heartdisease, ever_married, avg_glucose_level are the most correlated

In [ ]:
# singular plots


# hypertension
# heart_disease
# ever_married
# gender
# stroke
# smoking


sns.set_theme(style = 'darkgrid')
#plot_hypertension = sns.countplot(data = df, x = 'hypertension')
#plot_heart_disease = sns.countplot(data = df, x = 'heart_disease')
plot = sns.countplot(data = df, x = 'work_type')

In [ ]:
# For Avergae Glucose level
fig = plt.figure(figsize=(7,7))
sns.displot(df.avg_glucose_level, color="green", label="avg_glucose_level", kde= True)
plt.legend()

In [ ]:
# For BMI distribution
fig = plt.figure(figsize=(10,10))
sns.displot(df.bmi, color = 'blue', label = 'BMI', kde = True)
plt.legend()

In [ ]:
## Stroke vs No Stroke Avergae Glucose Level
plt.figure(figsize=(12,10))

sns.distplot(df[df['stroke'] == 0]["bmi"], color='green') # No Stroke - green
sns.distplot(df[df['stroke'] == 1]["bmi"], color='red') # Stroke - Red

plt.title('Stroke vs No Stroke BMI', fontsize = 15)
plt.xlim([10,100])
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(df[df['stroke']==0]['avg_glucose_level'], color = 'green')
sns.distplot(df[df['stroke'] == 1]['avg_glucose_level'], color = 'red')
plt.title('Stroke vs No stroke Average Glucose Level', fontsize = 15)
plt.xlim([30,300]) ## limit of x-axis
plt.show()

# Scatter Plot to see 3 variable variation

In [ ]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data = df, x = "age", y = "bmi", hue = "gender")
## to draw a line within the graph
graph.axhline(y = 25, linewidth = 4, color = "r", linestyle = '--')
graph.axhline()
plt.show()

In [ ]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data= df, x = "age", y = "avg_glucose_level", hue="gender")
graph.axhline(y = 150, linewidth = 4, color = "r", linestyle = "--")
fig.show()

According to the graph, all people above glucose level 150 and above BMI 25 are considered as having more risk to Stroke

In [ ]:
# With respect to smokers and non-smokers
df.head()

In [ ]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data = df, x = "smoking_status", y = "avg_glucose_level")
fig.show()

In [ ]:
# pair plot - all possible scatter plots
fig = plt.figure(figsize=(10,10))
sns.pairplot(df)
plt.show()

# Data Preprocessing

In [ ]:
# Getting predictor and the target variables separate
x = df.iloc[:, 1:-1].values
x

In [ ]:
y = df.iloc[:, -1].values
y

In [ ]:
## One hot encoding for categorical variables: gender, work_type, smoking_status
## Label encoding of binary variables: ever_married, residence_type

In [ ]:
df.rename(columns= {'Residence_type': 'residence_type'}, inplace=True)

In [ ]:
df.head()

In [ ]:
df['residence_type'].value_counts()

In [ ]:
# one hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

ct = ColumnTransformer(transformers= [('encoder', ohe, [0])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))
x

In [ ]:
x = np.delete(x, 2, 1) # deleting the 3rd column so as to delete one gender one hot encoded variable to prevent multicollinearity

In [ ]:
x[0][6]

In [ ]:
# one hot encoding of work_type
ct = ColumnTransformer(transformers = [('encoder', ohe, [6])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))
x

In [ ]:
print("This should be private: ", x[0][2])
print("This should also be private: ", x[2][2])
print("This should also be private: ", x[3][2])

In [ ]:
# Where does the work_type end
# 0,1: gender
# 2,3,4,5,6: work_type
x[0][7] # age

In [ ]:
# drop number 6 of work_type
x = np.delete(x, 6, 1)
x[0][6]

In [ ]:
# what is the current index of smoking status
x[0][13]

In [ ]:
# One hot encoding of smoking status
ct = ColumnTransformer(transformers = [('encoder', ohe, [13])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))
x

In [ ]:
# 0,1 : gender
# 2,3,4,5: work_type
# 6,7,8,9: smoking status
x = np.delete(x, 9, 1)
x.shape

# Final Tally:

### 0,1 index are gender
### 2,3,4,5 are work_type
### 6,7,8: smoking status

### Ellimination of one hot encoded variable per predictor is done to avoid Multicollineartity

In [ ]:
# Label Encoding
i,j = np.where(x == "Urban")

In [ ]:
i # row number

In [ ]:
j # Column number

In [ ]:
#Index 15 is ever_married and index 16 is residence_type
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 12] = le.fit_transform(x[:, 12])
x[:, 13] = le.fit_transform(x[:, 13])
x

In [ ]:
print("Shape x: ", x.shape) 
print("Shape y: ", y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
print("Number of records in training set: ", x_train.shape)
print("Shape of training set's labels: ", y_train.shape)
print("Number of records in test set: ", x_test.shape)
print("Shape of test set's labels: ", y_test.shape)

## Feature Scaling

##### StandardScaler does normalization (x-x_bar)/std_dev --> converts variables into normal dist. with mean 0 and standard dev = 1

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
x_test

## Handling Imbalanced Data using SMOTE

In [ ]:
print("Shape of training data: {} and {}".format(x_train.shape, y_train.shape))

### SMOTE is Synthetic Minority Oversampling Technique - overcoming the overfitting problem and class imbalance problem by generating synthetic minority records by random over-sampling

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 0)
x_train_res, y_train_res = sm.fit_resample(x_train, y_train.ravel())

In [ ]:
x_train_res.shape

In [ ]:
print("Shape of training data after synthetic oversampling: {} and {} ".format(x_train_res.shape, y_train_res.shape))

In [ ]:
print("Number of label 1 records before over_sampling: ", sum(y_train == 1))
print("Number of label 0 records before over_sampling: ", sum(y_train == 0))
print("\n")
print("Number of label 1 records before SMOTE: ", sum(y_train_res == 1))
print("Number of label 0 records after SMOTE: ", sum(y_train_res == 0))

In [ ]:
x_train_res

# Testing out Baseline Classification Models before Tuning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
# importing all metrics
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay,precision_score, recall_score, f1_score, classification_report, roc_auc_score, roc_curve, plot_roc_curve, precision_recall_curve, plot_precision_recall_curve, auc, average_precision_score

from sklearn.model_selection import cross_val_score

In [ ]:
models_array = []

## Adding it in an array so that could be made into a dataframe later
models_array.append(['Logistic Regression', LogisticRegression(random_state = 0)])
models_array.append(['K Nearest Neighbours', KNeighborsClassifier()])
models_array.append(['SVM Classifier', SVC(random_state=0)])
models_array.append(['Gaussian Naive Bayes', GaussianNB()])
models_array.append(['Bernoulli Naive Bayes', BernoulliNB()])
models_array.append(['Decision Tree Classifier', DecisionTreeClassifier(random_state = 0)])
models_array.append(['Random Forest', RandomForestClassifier(random_state = 0)])
models_array.append(['XGBoost Classifier', XGBClassifier(eval_metric = 'error',use_label_encoder=False)])

In [ ]:
list1= []

for m in range(len(models_array)):
    list2 = []
    model = models_array[m][1]
    model.fit(x_train_res, y_train_res)
    y_pred = model.predict(x_test)
    
    # confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # training accuracies - K-fold cross validation
    accuracies = cross_val_score(estimator = model, X = x_train_res, y = y_train_res, cv = 10)
    
    # Accuracy score of test set
    accuracy = accuracy_score(y_test, y_pred)
    
    # Area Score under Reciever Operating Characteristic Curve
    roc = roc_auc_score(y_test, y_pred) 
    
    # Precision
    precision = precision_score(y_test, y_pred)
    
    # Recall
    recall = recall_score(y_test, y_pred)
    
    # F1-score
    f1 = f1_score(y_test, y_pred)
    
    # Printing the results for every model
    print(models_array[m][0], ":")
    print("Training set mean accuracy: {:.2f} %".format(accuracies.mean()*100))
    print("Training set standard deviation: {:.2f} %".format(accuracies.std()*100))
    print("Test set accuracy: {:.2f} %".format(accuracy*100))
    print("Confusion Matrix: ", cm)
    print("Precision: {:.2f} %".format(precision*100))
    print("Recall: {:.2f} %".format(recall*100))
    print("F1 score: {:.2f}".format(f1))
    print("AUROC score: {:.2f}".format(roc))
    print("-----------------------------------------")
    print("\n")
    
    
    ## Adding all scores to a list so that we can form a dataframe later
    list2.append(models_array[m][0])
    list2.append(accuracies.mean()*100)
    list2.append(accuracies.std()*100)
    list2.append(accuracy)
    list2.append(precision)
    list2.append(recall)
    list2.append(f1)
    list2.append(roc)
    
    list1.append(list2) ## now list1 contains all the information of the results of the models - this can be put in a data frame
    
    

From above results we can see that on training and test set the following models performed well:
1. KNN
2. Decision Tree
3. Random Forest
4. XGBoost


The following performed poorly:
1. SVM
2. Logistics Regression
3. Naive Bayes (both Gausian and Bernoulli)

However, before eliminating any models, we should try to select the best hyperparameter and see the performance of each

In [ ]:
## Adding the results in a dataframe
df_results = pd.DataFrame(list1, columns = ['Model','Training Mean Accuracy', 'Training Standard Deviation', 'Test Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC Value'])
df_results.sort_values(by = ['Test Accuracy', 'Training Mean Accuracy', 'AUC Value'], inplace=True, ascending = False)
df_results

# Tuning Models

In [ ]:
# Grid search to select hyper-parameters 
from sklearn.model_selection import GridSearchCV

#### Hyperparameter C is used to prevent overfitting and provide regularization strength:
#### For more depth, I refered to this stackoverflow link: https://stackoverflow.com/questions/22851316/what-is-the-inverse-of-regularization-strength-in-logistic-regression-how-shoul


#### var_smoothing in Gaussian Naive Bayes artificially adds a user-defined standard deviation on how many samples to be included near or away from the mean in a gaussian distribution. Link: https://stackoverflow.com/questions/58046129/can-someone-give-a-good-math-stats-explanation-as-to-what-the-parameter-var-smoo


#### cv in grid search means the number of cross validation we have for each set of hyperparameters


In [ ]:
grid_models = [
    (LogisticRegression(), [{'C':[0.25, 0.5, 0.75, 1], 'random_state':[0]}]),
    #(KNeighborsClassifier(), [{'n_neighbors': [5,7,8,10], 'metric': ['euclidian', 'manhattan', 'chebyshev', 'minkowski']}]),
    (SVC(), [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear', 'rbf'], 'random_state':[0]}]),
    (GaussianNB(), [{'var_smoothing': [1e-09]}]),
    (BernoulliNB(), [{'alpha': [0.25, 0.5, 1]}]),
    (DecisionTreeClassifier(), [{'criterion': ['gini', 'entropy'], 'random_state': [0]}]),
    (RandomForestClassifier(), [{'n_estimators': [100, 150, 200], 'criterion': ['gini', 'entropy'], 'random_state': [0]}]),
    (XGBClassifier(), [{'learning_rate': [0.01, 0.05, 0.1], 'eval_metric': ['error'], 'use_label_encoder': [False]}])
]

In [ ]:
for i,j in grid_models:
    grid = GridSearchCV(estimator = i, param_grid = j, scoring = 'accuracy', cv = 10)
    grid.fit(x_train_res, y_train_res)
    best_accuracy = grid.best_score_
    best_param = grid.best_params_
    
    print("{}: ", i)
    print("Best Accuracy:  {:.2f} %".format(best_accuracy*100))
    print("Best parameters selected: ", best_param)
    print("------------------------------")
    print("\n")
    

# Random Forest and XGBoost after hyperparmarater tuning

## Random Forest

In [ ]:
# Fitting the random forest model with the above hyper parameters obtained by grid search

classifier = RandomForestClassifier(criterion = 'entropy', n_estimators = 150, random_state = 0)
classifier.fit(x_train_res, y_train_res)
y_pred = classifier.predict(x_test)
y_prob = classifier.predict_proba(x_test)[:,1] #probability with which the prediction is made
cm = confusion_matrix(y_test, y_pred)

print("predicted y: ", y_pred)

In [ ]:
cm

In [ ]:
rf_results_df = pd.DataFrame({'Predicted Stroke': y_pred, 
                             'Predicted Stroke Probability': y_prob,
                             'Actual Stroke': y_test})
rf_results_df.head()

In [ ]:
print(classification_report(y_test, y_pred))
print("ROC AUC Score: ", roc_auc_score(y_test, y_prob))
print("Test Accuracy: ", accuracy_score(y_test, y_pred))

In [ ]:
# Visualizing Confusion Matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,10))
sns.heatmap(cm, cmap = 'Blues', annot = True, fmt='d', linewidths = 5, cbar = False,
           annot_kws = {'fontsize': 15}, yticklabels= ['No Stroke', 'Stroke'], 
           xticklabels= ['Predicted No Stroke', 'Predicted Stroke'])
plt.yticks(rotation = 0)
plt.show()

In [ ]:
# Visualizing the ROC-AUC curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc_score = auc(false_positive_rate, true_positive_rate)
roc1 = roc_auc_score
print("AUC of tuned RF is: ", roc_auc_score)

In [ ]:
# Visualizing the ROC curve

sns.set_theme(style = 'white')
plt.figure(figsize = (10,10))
plt.plot(false_positive_rate, true_positive_rate, color='#b01717', label = 'AUC = %0.4f' %roc_auc_score)
plt.legend(loc = 'lower right')
plt.plot([0,1], [0,1], linestyle='--', color = '#174ab0')
plt.axis('tight') # to remove additional white space in the plot
plt.ylabel('True Positive Rate (Sensitivity)')
plt.xlabel('False Positive rate (1-Specificity)')
plt.legend()
plt.show()

## XG Boost with tuning

In [ ]:
classifier = XGBClassifier(eval_metric = 'error', use_label_encoder = False, learning_rate = 0.1)
classifier.fit(x_train_res, y_train_res)
y_pred = classifier.predict(x_test)
y_prob = classifier.predict_proba(x_test)[:,1]
cm = confusion_matrix(y_test, y_pred)

In [ ]:
# Putting the results in a dataframe
df_xgboost_results = pd.DataFrame({'Predicted Stroke Variable': y_pred, 
                                  'Predicted Stroke Probability': y_prob,
                                  'Actual Stroke Variable': y_test})
df_xgboost_results.head()

In [ ]:
cm

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# ROC AUC curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc_score = auc(false_positive_rate, true_positive_rate)
roc_auc_score

In [ ]:
## Visualization - Confusion Matrix
plt.figure(figsize = (10,10))
sns.heatmap(cm, cmap = "Blues", annot = True, fmt = 'd', linewidth = 5, cbar = False, annot_kws = {'fontsize': 15},
           xticklabels = ['Predicted No Stroke', 'Predicted Stroke'], yticklabels = ['No Stroke', 'Stroke'])
plt.show()

In [ ]:
## Visualizaton - ROC AUC Curve

sns.set_theme(style = 'white')
plt.figure(figsize = (10,10))
plt.plot(false_positive_rate, true_positive_rate, color='#b01717', label = 'AUC = %0.4f' %roc_auc_score)
plt.legend(loc = 'lower right')
plt.plot([0,1], [0,1], linestyle='--', color = '#174ab0')
plt.axis('tight') # to remove additional white space in the plot
plt.ylabel('True Positive Rate (Sensitivity)')
plt.xlabel('False Positive rate (1-Specificity)')
plt.legend()
plt.show()

# Keras ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.regularizers import l2

In [ ]:
# Building the ANN with 3 dense layers
import tensorflow as tf

def ann_classifier():
    ann = tf.keras.models.Sequential() ## Sequential layer has one input tensor and one output tensor
    
    # Dense layer 1
    ann.add(tf.keras.layers.Dense(units=8, kernel_regularizer = l2(0.01), bias_regularizer = l2(0.01), activation = 'relu'))
    
    #Dense layer 2
    ann.add(tf.keras.layers.Dense(units=8, kernel_regularizer = l2(0.01), bias_regularizer = l2(0.01), activation = 'relu'))
    
    # Dense layer 3
    ann.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
    
    
    ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return ann

In [ ]:
model = KerasClassifier(build_fn = ann_classifier, batch_size = 32, epochs = 50)

In [ ]:
## Evaluating the basic untuned ANN
accuracies = cross_val_score(estimator = model, X=x_train_res, y=y_train_res, cv = 5)

In [ ]:
## Checking the meand and std dev of the accuracies obtained
mean = accuracies.mean()
mean

In [ ]:
std_dev = accuracies.std()
std_dev

## Tuning ANN using GridSearch

In [ ]:
def ann_classifier(optimizer = 'adam'):
    ann = tf.keras.models.Sequential() ## Sequential layer has one input tensor and one output tensor
    
    # Dense layer 1
    ann.add(tf.keras.layers.Dense(units=8, kernel_regularizer = l2(0.01), bias_regularizer = l2(0.01), activation = 'relu'))
    
    #Dense layer 2
    ann.add(tf.keras.layers.Dense(units=8, kernel_regularizer = l2(0.01), bias_regularizer = l2(0.01), activation = 'relu'))
    
    # Adding dropout which is a technique to randomly ignore neurons while training to reduce overfitting
    tf.keras.layers.Dropout(0.6)
     
    # Dense layer 3
    ann.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
    
    
    ann.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return ann

In [ ]:
model = KerasClassifier(build_fn = ann_classifier, batch_size = 32, epochs = 50)

parameters = {
    'batch_size': [25, 32],
    'epochs': [50,100,150],
    'optimizer': ['adam', 'rmsprop']
}


grid_search = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grid_search.fit(x_train_res, y_train_res)

In [ ]:
best_accuracy = grid_search.best_score_
best_params = grid_search.best_params_

print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters: ", best_params)


# Final Tuned ANN Model

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 32, kernel_regularizer = l2(0.01), bias_regularizer = l2(0.01), activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 32, kernel_regularizer = l2(0.01), bias_regularizer = l2(0.01), activation = 'relu'))
tf.keras.layers.Dropout(0.6)
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

ann.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

model = ann.fit(x_train_res, y_train_res, batch_size = 25, epochs = 150, validation_split = 0.2)

In [ ]:
# Loss Graph
# Here model is actually just the ann history


loss_training = model.history['loss']
loss_validation = model.history['val_loss']
epochs = range(1,151)

plt.plot(epochs, loss_training, 'g', label = 'Training Loss')
plt.plot(epochs, loss_validation, 'b', label = 'Validation Loss')

plt.title('Training and Validation Loss of Tuned ANN')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.show()

In [ ]:
# Training and Validation accuracy plot
training_accuracy = model.history['accuracy']
validation_accuracy = model.history['val_accuracy']
epochs = range(1,151)

plt.plot(epochs, training_accuracy, 'g', label= 'Training Accuracy')
plt.plot(epochs, validation_accuracy, 'b', label = 'Validation Accuracy')
plt.title('Training and Validation Accuracy for Tuned ANN')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
y_pred = ann.predict(x_test)

In [ ]:
def threshold_prediction(y_pred):
    y_pred_result = []
    for i in y_pred:
        if i > 0.5:
            y_pred_result.append(1)
        else:
            y_pred_result.append(0)
    return y_pred_result
            
y_pred_result = threshold_prediction(y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred_result)
cm

In [ ]:
accuracy_test = accuracy_score(y_test, y_pred_result)
accuracy_test

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc_score = auc(false_positive_rate, true_positive_rate)
roc1 = roc_auc_score
print("AUC of tuned ANN is: ", roc_auc_score)

In [ ]:
plt.figure(figsize=(8,5))
sns.heatmap(cm, cmap = "Blues", annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws ={'fontsize': 15}, yticklabels = ['No Stroke', 'Stroke'], xticklabels = ['Predicted No Stroke', 'Predicted Stroke'])
plt.show()

# Summary of Results

Clearly the 3 good performing models for Stroke Prediction are as follows:
1. Random Forest: accuracy: 0.89 , auc: 0.75
2. XG Boost: accuracy: 0.87 , auc: 0.74
3. ANN: accuracy: 0.85 , auc: 0.73

So a tuned Random Forest performed best for this data